In [5]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [6]:
from coco import CocoCaptionsDataset, CocoDataModule, ImageCaptioningModel, sweep_iteration, collate_fn, sweep, Config
from adapter_utils import AdapterConf, FTConf, PartialFTConf, RNNAdapterConf
import pytorch_lightning as pl


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
sweep_config = {
    "method": "bayes",
    "name": "adapter_64",
    "metric": {
        "name": "val_loss",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "min": 1e-6,
            "max": 1e-3
        },
    },
}
sweep(sweep_config)

wandb: Agent Starting Run: b6rmt5vm with config:
wandb: 	learning_rate: 0.0004969205711931511


Starting sweep iteration


wandb: Currently logged in as: ngjhn (read). Use `wandb login --relogin` to force relogin


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were n

Enables adapter tuning; num trainable params 2,416,128, ratio 0.011
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(


loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /data/home/ngjhn/read/lightning_logs/adapter exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | VisionEncoderDecoderModel | 226 M 
1 | val_loss  | AvgMetric                 | 0     
2 | test_loss | AvgMetric                 | 0     
--------------------------------------------------------
2.4 M     Trainable params
224 M     Non-trainable params
226 M     Total params
906.746   Total estimated model params size (MB)


Epoch 0:   1%|          | 16/2587 [02:21<6:18:53,  8.84s/it, v_num=t5vm, loss=5.710]

In [ ]:
data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=32,
    val_batch_size=512)
data_module.setup(stage="test")


In [ ]:
model = ImageCaptioningModel(
    "google/vit-base-patch16-224-in21k",
    "bert-base-uncased",
    data_module=data_module,
    learning_rate=1e-6,
    peft_conf=peft_confg,
)

In [ ]:
model.load_from_checkpoint(
    "/data/home/ngjhn/read/lightning_logs/rnn/adapter-epoch=17-val_loss=1.26.ckpt",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    text_model_name_or_path="bert-base-uncased",
    data_module=data_module,
    learning_rate=1e-6,
    peft_conf=peft_confg,
)


In [5]:
!unset $LD_LIBRARY_PATH 

/bin/bash: line 0: unset: `/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:': not a valid identifier


In [ ]:
model.evaluate("cuda:0")